# Samples 10-kernel-search-internet-agent

Source : 

In [22]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.14.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.14.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.14.1-alpha"
#r "nuget: dotenv.net"

Installed Packages dotenv.net, 3.1.3 Microsoft.SemanticKernel, 1.14.1 Microsoft.SemanticKernel.Plugins.Core, 1.14.1-alpha Microsoft.SemanticKernel.Plugins.Web, 1.14.1-alpha

In [25]:
#r "nuget: Microsoft.Extensions.Logging"
#r "nuget: Microsoft.Extensions.Logging.Abstractions"
#r "nuget: Microsoft.Extensions.Logging.Console"
#r "nuget: Microsoft.Extensions.Logging.Debug"
#r "nuget: Microsoft.Extensions.DependencyInjection"

Installed Packages microsoft.extensions.dependencyinjection, 8.0.0 microsoft.extensions.logging, 8.0.0 microsoft.extensions.logging.abstractions, 8.0.1 Microsoft.Extensions.Logging.Console, 8.0.0 Microsoft.Extensions.Logging.Debug, 8.0.0

## Instantiate the kernel


In [27]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

//Create Kernel builder
var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

Console.WriteLine($"Use Azure Open AI : {useAzureOpenAI}");
Console.WriteLine($"Model : {model}");
Console.WriteLine($"Azure EndPoint : {azureEndpoint}");
// Console.WriteLine($"API Key : {apiKey}");

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);


// Logger
builder.Services.AddLogging(c => c.AddDebug().SetMinimumLevel(LogLevel.Trace));


var kernel = builder.Build();

Use Azure Open AI : True
Model : gpt-35-turbo-16k
Azure EndPoint : https://oai-ygo74-switzerland.openai.azure.com/


In [28]:
using Microsoft.SemanticKernel.Plugins.Web.Google;
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;


In [29]:
#pragma warning disable SKEXP0001, SKEXP0010, SKEXP0020, SKEXP0050

using dotenv.net;
// var options = new DotEnvOptions(envFilePaths: new String[] {@"./config"});

dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

string googleApiKey=env[ "GOOGLE_SEARCH_API_KEY" ];
string googleSearchEngineId=env[ "GOOGLE_SEARCH_CENTER_CONNECTION_ID" ];

var googleConnector = new Microsoft.SemanticKernel.Plugins.Web.Google.GoogleConnector(
        apiKey: googleApiKey,
        searchEngineId: googleSearchEngineId);

var google = new WebSearchEnginePlugin(googleConnector);
kernel.ImportPluginFromObject(new WebSearchEnginePlugin(googleConnector), "google");

In [30]:
var internet_searcher  = """
You are a world class researcher, who can detailed research on any topic and produce facts based results;
You do not make things up, you will try as hard as possible to gather facts and data to back up the research.

You have the functions google_search to find elements on the web.
You have the functions scrape_website to rerieve content of a page.
if you have to search a person, use simple quote for the first name and last name. If you want to limit a query to a specific site, use site keyword to limit the search to a specific site

Please make sure you complete the objective above with the following rules:
1. You should do enough research to gather as much information as possible about the objective
2. If there are url of relevant links and articles, you will scrape it with the scrape_website function to gather more information
3. After scraping and search, you should think "Is there any new things I should search and scraping based on the data I collected to increase research quality? If answer is yes, continue; But don' do this more than 3 iterations.
4. You should not make things up, you should only write facts and data that you gathered
5. In the final output, You should include all reference data and links to back up your research.
6. Do not use G2, they are mostly out dated data

SAY TERMINATE when you found the result.

---
Here are expected examples :

Example :
question : search devops on github
query : devops site:github.com

Example :
question : find contribution for code from Yannick GOBERT a devops engineer on internet, he can be known with his alias ygo or ygo74
query : contribution for code from 'Yannick GOBERT' devops engineer

Example :
question : find contribution for code from Yannick GOBERT a devops engineer on internet, he can be known with his alias ygo or ygo74
query : contribution for code from 'YGO74' devops engineer


""";

In [31]:
// Retrieve the chat completion service from the kernel
IChatCompletionService chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();


In [32]:
ChatHistory chatMessages = new ChatHistory();
chatMessages.AddSystemMessage(internet_searcher);


In [33]:
var question = "Qui a gané la ligue des champions de football en 2024";
chatMessages.AddUserMessage(question);

// Get the chat completions
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new()
{
    ToolCallBehavior  = ToolCallBehavior.AutoInvokeKernelFunctions
};

var result = chatCompletionService.GetStreamingChatMessageContentsAsync(
    chatMessages,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel);

In [34]:
// Stream the results
    string fullMessage = "";
    await foreach (var content in result)
    {
        if (content.Role.HasValue)
        {
            System.Console.Write("Assistant > ");
        }
        System.Console.Write(content.Content);
        fullMessage += content.Content;
    }
    System.Console.WriteLine();
    chatMessages.AddAssistantMessage(fullMessage);

Assistant > Assistant > Je suis désolé, mais je ne suis pas capable de prédire l'avenir. La ligue des champions de football de 2024 n'a pas encore eu lieu, donc je ne peux pas vous dire qui en sera le vainqueur.
